<center>

<div style="text-align:center"><img src="_static/small_e_logo_cropped.png" width="40%" /></div>
<div style="text-align:center"><img src="_static/pangeo_simple_logo.png" width="175px" /></div>
</center>

Pangeo Tools
===========




A brief overview of the ecosystem of tools used by the [Pangeo Project](http://pangeo.io/).

In [ ]:
%matplotlib inline
import numpy as np
import xarray as xr
import warnings

warnings.simplefilter("ignore", )
np.set_printoptions(precision=4, threshold=5)
xr.set_options(display_width=80)

### Pangeo is:

- a community promoting open, reproducible, and scalable science.
- an integrated ecosystem of open source software tools.
- a community platform for Big Data Geoscience.

### Where to find Pangeo:

- Online: http://pangeo.io/
- GitHub: https://github.com/pangeo-data/
- Discourse: https://discourse.pangeo.io/

<div style="text-align:center"><img src="_static/scientific_python_eco.png" width="100%" /></div>


## The Basics

NumPy / SciPy / Pandas/ Jupyter


### NumPy

NumPy is the fundamental package for scientific computing with Python. It contains among other things:

- a powerful N-dimensional array object
- sophisticated (broadcasting) functions
- tools for integrating C/C++ and Fortran code
- useful linear algebra, Fourier transform, and random number capabilities

In [ ]:
import numpy as np

x = np.ones((4, 2))
x.sum(axis=1)

### SciPy

SciPy is a collection of mathematical algorithms and convenience functions built on the NumPy extension of Python. It contains subpackages that cover:

- clustering
- FFTs
- interpolation
- linear algebra
- singal processing
- stats
- optimization

In [ ]:
from scipy import spatial
x, y = np.mgrid[0:5, 2:8]
tree = spatial.KDTree(list(zip(x.ravel(), y.ravel())))
pts = np.array([[0, 0], [2.1, 2.9]])
tree.query(pts)

tree.query(pts[0])

### Pandas

Pandas is a Python package providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive. Pandas is well suited for:

- Tabular data types (see `Series` and `DataFrame` objects)
- Timeseries data manipulation like resampling
- Database-like operations like aligning, merging, joining, reshaping, and grouping

In [ ]:
import pandas as pd

df = pd.read_csv('./data/chico_temperature.csv', index_col=0, parse_dates=True)
display(df.head())
df.resample('AS').max().plot()

## Jupyter

TODO:

- JupyterHub
- Jupyterlab

## Xarray

<!-- <div style="text-align:center"><img src="_static/dataset-diagram.png" width="50%" /></div> -->
<img src="_static/dataset-diagram.png" align="right" width=66% alt="Xarray Dataset">


Xarray is a Python library that provides data structures and tools for working with multidimensional labeled datasets and arrays. Xarray enables users to perform operations on complex datasets making it a powerful high-level tool for data analysis. 

- Inspired by Pandas and NetCDF
- Labeled N-Dimensional data structures (`DataArray` and `Dataset`)
- Toolkit for data manipulation and visualization
- Integrates with scientific Python ecosystem (Pandas/Matplotlib/Dask/etc)
- Backend support for a wide range of ND data formats (NetCDF, GRIB, Raster, Zarr)


#### xarray.Dataset

In [ ]:
import xarray as xr
ds = xr.tutorial.load_dataset('air_temperature')
ds

#### xarray.DataArray

In [ ]:
da = ds['air']

da.isel(time=0, lat=5, lon=10)  # select using integer index
da.sel(time='2013-01-01T18:00:00', lat=62.5, lon=225.0)  # select based on label

In [ ]:
# resample data to monthly means
da_month = da.resample(time='MS').mean('time')

da_climo = da_month.groupby('time.month').mean('time')
da_climo
# remove the monthly climotology
da_no_climo = da_month.groupby('time.month') - da_climo
da_no_climo

In [ ]:
# integrated plotting
da_climo.plot(col='month', col_wrap=6, robust=True)

## Dask

<img src="https://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg" align="right" width=50% alt="Dask Logo">

Dask is a flexible parallel computing library for analytic computing. Dask provides dynamic parallel task scheduling and high-level big-data collections like dask.array and dask.dataframe.

- Dask Array implements a subset of the NumPy ndarray interface using blocked algorithms, cutting up the large array into many small arrays.
- Tools like Xarray and Iris use Dask arrays under-the-hood
- Dask can be deployed on a local computer, HPC, or the Cloud

## Dask Arrays

A dask array looks and feels a lot like a numpy array. However, a dask array doesn't directly hold any data. Instead, it symbolically represents the computations needed to generate the data. Nothing is actually computed until the actual numerical values are needed. This mode of operation is called "lazy"; it allows one to build up complex, large calculations symbolically before turning them over the scheduler for execution.

In [ ]:
# Numpy
import numpy as np
shape = (1000, 4000)
ones_np = np.ones(shape)
ones_np

In [ ]:
# Dask
import dask.array as da
chunk_shape = (1000, 1000)
ones = da.ones(shape, chunks=chunk_shape)
ones

### Deploying Dask

The Dask Schedulers orchestrate the tasks in the Task Graphs so that they can be run in parallel. How they run in parallel, though, is determined by which Scheduler you choose.

There are 3 *local* schedulers:

- **Single-Thread Local**: For debugging, profiling, and diagnosing issues
- **Multi-threaded**: Using the Python built-in threading package (the default for all Dask operations except Bags)
- **Multi-process**: Using the Python built-in multiprocessing package (the default for Dask Bags)

and 1 distributed scheduler, which we will talk about later:

- **Distributed**: Using the dask.distributed module (which uses tornado for TCP communication). The distributed scheduler uses a Cluster to manage communication between the scheduler and the "workers". This is described in the next section.

### Distributed Clusters (http://distributed.dask.org/)¶
Dask can be deployed on distributed infrastructure, such as a an HPC system or a cloud computing system.

- LocalCluster - Creates a Cluster that can be executed locally. Each Cluster includes a Scheduler and Workers.
- Client - Connects to and drives computation on a distributed Cluster

**Dask Jobqueue (http://jobqueue.dask.org/)**
- PBSCluster
- SlurmCluster
- etc.

**Dask Kubernetes (http://kubernetes.dask.org/)**
- KubeCluster

In [ ]:
from dask.distributed import Client
# On Cheyenne:
from dask_jobqueue import PBSCluster
cluster = PBSCluster(project=...)

# On Caseper
from dask_jobqueue import SlurmCluster
cluster = SlurmCluster(project=...)

# On the cloud
from dask_kubernetes import KubeCluster
cluster = KubeCluster()

# use adaptive scaling!
cluster.adapt(0, 30)
client = Client(cluster)

## Visualization

Python a rich ecosystem of data visualization tools.

- Matplotlib:
- Holoviz (Bokeh, Hvplot, Datashader, Panel):
- Cartopy:

More coming here soon.

## Data Catalogs